In [59]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [60]:
!pip install nbformat nbconvert
try:
  from fastai.gen_doc.nbdoc import *
except Exception as e:
  print(e)

No module named 'fastai.gen_doc'


# Building stuff from ground up

We will be building everything from scratch only using basic indexing into a tensor

and see what exactly what is happening in loss.backward

## Building a neural net from scratch

modelling a neuron : a neuron is shaped as out = sum of (xiwi) + b



In [61]:
inputs =[0,0,0]
weights =[0,0,0]
bias = 0
output = sum([x*w for x,w in zip(inputs, weights)]) + bias

THis is then sent to an activation function Relu for example


In [62]:
def relu(x):
  return x if x>=0 else 0

Deep learning model is then bilt by stacking a lot of those nerurons in successive layer with a certain number of neurons  and linking the output and inputs ato each neurtoron. it is called a dense layer or densly connectedor linear layer

In [63]:
sum([x*w for x,w in zip(inputs, weights)]) + bias

0

we are essentially doing matrix multiplication here

x is supposed to hace batch_Size by n_inputs

while w would be n_neurons by n_inputs

so 

In [64]:
input = [[0],[0],[0]]
weights = [[0,0,0]] 
def transpose(weights):
  return weights

try:
  y =  inputs @  transpose(weights)+ b
except Exception as e:
  print(e)

unsupported operand type(s) for @: 'list' and 'list'


the output then should be if size batch_size by neurons and in position (i,j)

In [65]:
x = inputs
w = weights
for i in range(0, 2):
  for j in range(0,2):
    try:
      y[i,j] = sum([a*b for a,b in zip(x[i,:],w.t[j,:])]) + b[j]
    except:
      pass

## doing matirx multi



In [66]:
import torch
from torch import tensor

In [67]:
def matmul(a,b):
  ar, ac = a.shape
  br, bc = b.shape
  assert ac==br
  c= torch.zeros(ar,bc)
  for i in range(ar):
    for j in range(bc):
      for k in range(ac):
        c[i,j] += a[i,k] * b[k,j]
  return c

In [68]:
m1 = torch.randn(5,28*28)
m2 = torch.randn(784,10)

In [69]:
# lets see our time
%time t1 = matmul(m1, m2)

CPU times: user 1.07 s, sys: 0 ns, total: 1.07 s
Wall time: 1.08 s


In [70]:
# compare it with time of pytorch built in @
%timeit -n 20 t2=m1@m2

20 loops, best of 3: 7.76 µs per loop


How come pytorch so fast? it used C++.
we need to vectorize the computations tensor so that we can take advantage of speed of pytorch

### Element wise broadcasting

all these are element wise operations

In [71]:
a = tensor([10., 6, -4])
b = tensor([2., 8, 7])
a+b

tensor([12., 14.,  3.])

In [72]:
a<b

tensor([False,  True,  True])

In [73]:
(a<b).all(), (a==b).all()

(tensor(False), tensor(False))

In [74]:
(a+b).mean()

tensor(9.6667)

In [75]:
(a+b).mean().item()

# item converts into a normal boolean number

9.666666984558105

In [76]:
# elementwise cannot be done on different shaped tesnor without broadcasting
m = tensor([[0,0,0]])
n = tensor([[0,0]])

try:
  m*n
except Exception as e:
  print(e)

The size of tensor a (3) must match the size of tensor b (2) at non-singleton dimension 1


With elementwise arithmetic, we can remove one of our three nested loops: we can multiply the tensors that correspond to the `i`-th row of `a` and the `j`-th column of `b` before summing all the elements, which will speed things up because the inner loop will now be executed by PyTorch at C speed. 

To access one column or row, we can simply write `a[i,:]` or `b[:,j]`. The `:` means take everything in that dimension. We could restrict this and take only a slice of that particular dimension by passing a range, like `1:5`, instead of just `:`. In that case, we would take the elements in columns or rows 1 to 4 (the second number is noninclusive). 

One simplification is that we can always omit a trailing colon, so `a[i,:]` can be abbreviated to `a[i]`. With all of that in mind, we can write a new version of our matrix multiplication:

In [77]:
def matmul(a,b):
  ar,ac = a.shape
  br, bc = b.shape
  assert ac==br
  c = torch.zeros(ar, bc)
  for i in range(ar):
    for j in range(bc):
      c[i,j] = (a[i] * b[:,j]).sum()
  return c

In [78]:
m1

tensor([[ 0.7415,  0.7774, -0.8424,  ..., -0.3819, -1.9647,  0.4475],
        [-1.7090,  0.4627, -0.6896,  ...,  1.3506,  1.0114, -0.8189],
        [ 1.3658,  0.7219, -1.0544,  ..., -0.7875, -2.2777,  1.1233],
        [-1.1400,  0.8963,  0.3066,  ..., -2.2176, -0.0887,  0.5459],
        [-1.2235, -0.3458, -1.2032,  ..., -0.6197,  0.0967,  0.4876]])

In [79]:
m2

tensor([[ 0.9013, -0.6256, -0.5285,  ..., -0.5408,  0.7302, -1.1211],
        [ 1.0788,  0.1453,  0.1452,  ...,  0.8222, -0.2700,  0.5019],
        [ 1.0633, -0.6268,  0.5282,  ...,  0.6808, -0.9093, -0.6016],
        ...,
        [-0.0309,  1.0257, -1.0388,  ...,  0.2959,  0.5726,  0.7132],
        [-0.2292,  0.7344, -1.9675,  ..., -0.4090,  0.5380, -0.1674],
        [-0.1377, -0.4448,  0.7137,  ..., -1.3121,  0.1527,  0.2252]])

In [80]:
%time t3 = matmul(m1,m2)

CPU times: user 1.78 ms, sys: 0 ns, total: 1.78 ms
Wall time: 3.88 ms


### Broadcasting

Broadcasting gives specific rules to codify when shapes are compatible when trying to do an elementwise operation, and how the tensor of the smaller shape is expanded to match the tensor of the bigger shape. It's essential to master those rules if you want to be able to write code that executes quickly. In this section, we'll expand our previous treatment of broadcasting to understand these rules.

In [81]:
# broadcasting with a scalar

a= tensor([10., 6, -4])
a > 0

tensor([ True,  True, False])

In [82]:
m = tensor([[1.,2,3], [4,5,6], [7,8,9]])
(m-5)/2.73

tensor([[-1.4652, -1.0989, -0.7326],
        [-0.3663,  0.0000,  0.3663],
        [ 0.7326,  1.0989,  1.4652]])

In [83]:
c = tensor([10.,20,30])
m = tensor([[1.,2,3],[4,5,6],[7,8,9]])
m.shape, c.shape

(torch.Size([3, 3]), torch.Size([3]))

In [84]:
m + c

tensor([[11., 22., 33.],
        [14., 25., 36.],
        [17., 28., 39.]])

In [85]:
# behind the scenes

c.expand_as(m)

tensor([[10., 20., 30.],
        [10., 20., 30.],
        [10., 20., 30.]])

In [86]:
t = c.expand_as(m)
t.storage()

 10.0
 20.0
 30.0
[torch.FloatStorage of size 3]

In [87]:
t.stride(), t.shape

((0, 1), torch.Size([3, 3]))

In [88]:
c = tensor([10.,20])
m = tensor([[1.,2,3],[4,5,6]])
try:
  c+m
except Exception as e:
  print(e)

The size of tensor a (2) must match the size of tensor b (3) at non-singleton dimension 1


In [89]:
c = tensor([10.,20,30])
m = tensor([[1.,2,3],[4,5,6],[7,8,9]])
c = c.unsqueeze(1)
m.shape, c.shape

(torch.Size([3, 3]), torch.Size([3, 1]))

In [90]:
c + m

tensor([[11., 12., 13.],
        [24., 25., 26.],
        [37., 38., 39.]])

In [91]:
t = c.expand_as(m)
t.storage()

 10.0
 20.0
 30.0
[torch.FloatStorage of size 3]

In [92]:
t.stride(), t.shape

((1, 0), torch.Size([3, 3]))

In [93]:
c = tensor([10.,20,30])
c.shape,c.unsqueeze(0).shape,c.unsqueeze(1).shape

(torch.Size([3]), torch.Size([1, 3]), torch.Size([3, 1]))

In [94]:
c.shape, c.unsqueeze(0).shape, c.unsqueeze(1).shape

(torch.Size([3]), torch.Size([1, 3]), torch.Size([3, 1]))

In [95]:
c.shape, c[None,:].shape, c[:,None].shape

(torch.Size([3]), torch.Size([1, 3]), torch.Size([3, 1]))

In [96]:
c[None].shape,c[..., None].shape

(torch.Size([1, 3]), torch.Size([3, 1]))

In [97]:
def matmul(a,b):
  ar,ac = a.shape
  br,bc = b.shape
  assert ac == br
  c = torch.zeros(ar, bc)
  for i in range(ar):
    c[i] = (a[i].unsqueeze(-1)*b).sum(dim=0)
  return c

In [98]:
%timeit -n 20 t4 = matmul(m1,m2)

20 loops, best of 3: 473 µs per loop


### Broadcasting Rules

Arrays do not need to have the same number of dimensions. For example, if you have a 256×256×3 array of RGB values, and you want to scale each color in the image by a different value, you can multiply the image by a one-dimensional array with three values. Lining up the sizes of the trailing axes of these arrays according to the broadcast rules, shows that they are compatible:

```
Image  (3d tensor): 256 x 256 x 3
Scale  (1d tensor):  (1)   (1)  3
Result (3d tensor): 256 x 256 x 3
```
    
However, a 2D tensor of size 256×256 isn't compatible with our image:

```
Image  (3d tensor): 256 x 256 x   3
Scale  (1d tensor):  (1)  256 x 256
Error
```

In our earlier examples we had with a 3×3 matrix and a vector of size 3, broadcasting was done on the rows:

```
Matrix (2d tensor):   3 x 3
Vector (1d tensor): (1)   3
Result (2d tensor):   3 x 3
```

As an exercise, try to determine what dimensions to add (and where) when you need to normalize a batch of images of size `64 x 3 x 256 x 256` with vectors of three elements (one for the mean and one for the standard deviation).

### Einstein summmation

This is a compact representation for combining products and sums in general way

`ik, kj -> ij`

In [99]:
def matmul(a,b):
  return torch.einsum('ik, kj->ij', a, b)

In [100]:
try:
  torch.einsum('bik, bkj->bij',a,b)
except Exception as e:
  print(e)

dimension mismatch for operand 0: equation 3 tensor 1


In [101]:
%timeit -n 20 t5 = matmul(m1, m2)

20 loops, best of 3: 87 µs per loop


## Forward and Backward Passes



In [103]:
# defining and initializing a layer

def lin(x, w, b):
  return x @w + b

# the activationfunction we will use will be relu



In [102]:
# lets take random tesnors as input and targets

x = torch.randn(200,100)
y = torch.randn(200)

In [104]:
# for out two layer model we will need two weight matricss 
# and two bias vectors
# we initiliaze the weights randomly and the bias at zero

w1 = torch.randn(100,50)
b1 = torch.zeros(50)
w2 = torch.randn(50,1)
b2 = torch.zeros(1)

In [105]:
l1 = lin(x, w1, b1)
l1.shape

torch.Size([200, 50])

In [106]:
l1.mean(), l1.std()

# there is a problem here though themean is clos to zero
# however the standard deviation is from 1 to 10
# it means they will scale the number by 10 everyt time


(tensor(-0.0778), tensor(10.1446))

In [108]:
x = torch.randn(200,100)
for i in range(50):
  x = x @torch.randn(100,100)
x[0:5,0:5]

# if we do multiply 50 times x then the number is too big so it gives nan

tensor([[nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan]])

In [109]:
x = torch.randn(200,100)
for i in range(50):
  x = x @ (torch.randn(100,100) * 0.01)
x[0:5,0:5]

# if we use too small weights then we would be left with 0 everywhere

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

1. We can compute the exact value to use mathematically, as illustrated by Xavier Glorot and Yoshua Bengio in ["Understanding the Difficulty of Training Deep Feedforward Neural Networks"](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf). The right scale for a given layer is $1/\sqrt{n_{in}}$, where $n_{in}$ represents the number of inputs.

In [111]:
# therefore we need to scale our weights matrics so 
# that the standard deviation remains one
# since we have 100 inputswe scale it by 0.1

x = torch.randn(200,100)
for i in range(50):
  x = x @ (torch.randn(100,100) * 0.1)
x[0:5, 0:5]


tensor([[-0.8013, -0.1897, -0.4023, -0.4735,  0.1845],
        [ 0.2788, -0.1554,  0.8431,  0.1736, -0.5525],
        [ 0.0986,  0.4494,  0.1055,  0.3769, -0.0502],
        [ 0.9886, -0.3836, -0.2021,  0.1302, -0.4169],
        [-1.0789,  0.4563,  0.0734, -0.1716,  0.6076]])

In [112]:
x.std()

tensor(0.9380)

In [113]:
# going back to our neural network

x = torch.randn(200,100)
y = torch.randn(200)

In [114]:
# for our weights we will use the right scale 
# thisis Xavier initialization

from math import sqrt
w1 = torch.randn(100,50)/sqrt(100)
b1 = torch.zeros(50)
w2 = torch.randn(50,1)/sqrt(50)
b2 = torch.zeros(1)

In [115]:
l1 = lin(x, w1, b1)
l1.mean(), l1.std()

(tensor(-0.0078), tensor(1.0302))

In [116]:
# now that we have mean and e=deviation under control
# we have defined a relu
#relu removes negatives and clamps at 0

def relu(x):
  return x.clamp_min(0.)

In [117]:
l2 = relu(l1)
l2.mean(), l2.std()

# this is too less

(tensor(0.4058), tensor(0.6044))

In [118]:
x = torch.randn(200,100)
for i in range(50):
  x = relu(x @ (torch.randn(100,100)*0.1))
x[0:5,0:5]

tensor([[9.5434e-09, 0.0000e+00, 1.0853e-08, 0.0000e+00, 7.4718e-09],
        [4.3009e-09, 0.0000e+00, 1.5764e-09, 0.0000e+00, 7.0184e-09],
        [3.9585e-09, 0.0000e+00, 7.0260e-09, 0.0000e+00, 1.9430e-09],
        [6.6478e-09, 0.0000e+00, 2.5971e-09, 0.0000e+00, 1.0579e-08],
        [1.2210e-08, 0.0000e+00, 5.3824e-10, 0.0000e+00, 1.7850e-08]])

In [119]:
x = torch.randn(200,100)
for i in range(50): x = relu(x@ (torch.randn(100,100) *sqrt(2/100)))
x[0:5, 0:5]

tensor([[0.7071, 0.0000, 2.1879, 0.3887, 0.0000],
        [0.5339, 0.0000, 1.4717, 0.4260, 0.0000],
        [0.4231, 0.0000, 1.2220, 0.3425, 0.0000],
        [0.6724, 0.0000, 2.0828, 0.3392, 0.0000],
        [0.8956, 0.0000, 2.5786, 0.6576, 0.0000]])

In [120]:
x =  torch.randn(200,100)
y = torch.randn(200)

In [121]:
w1= torch.randn(100,50) * sqrt(2/100)
b1 = torch.randn(50) * sqrt(2/50)
w2 = torch.randn(50,1) * sqrt(2/50)
b2 = torch.zeros(1)

In [122]:
l1 = lin(x, w1, b1)
l2 = relu(l1)
l2.mean(), l2.std()

#much better

(tensor(0.5606), tensor(0.8227))

In [123]:
# letss define the model
# this is the forwrd pass
def model(x):
  l1 = lin(x, w1, b1)
  l2 = relu(l1)
  l3 = lin(l2,w2,b2)
  return l3 

In [124]:
out = model(x)
out.shape

torch.Size([200, 1])

In [126]:
# we can get rid of this trailing 1 dimension by using the
# squeeze function

def mse(output, targ):
  return (output.squeeze(-1) - targ).pow(2).mean()

# this is our loss
loss = mse(out,y)

## gradients and backward pass

We've seen that PyTorch computes all the gradients we need with a magic call to `loss.backward`, but let's explore what's happening behind the scenes.

Now comes the part where we need to compute the gradients of the loss with respect to all the weights of our model, so all the floats in `w1`, `b1`, `w2`, and `b2`. For this, we will need a bit of math—specifically the *chain rule*. This is the rule of calculus that guides how we can compute the derivative of a composed function:

$$(g \circ f)'(x) = g'(f(x)) f'(x)$$





Jeremy: I find this notation very hard to wrap my head around, so instead I like to think of it as: if `y = g(u)` and `u=f(x)`; then `dy/dx = dy/du * du/dx`. The two notations mean the same thing, so use whatever works for you.

Our loss is a big composition of different functions: mean squared error (which is in turn the composition of a mean and a power of two), the second linear layer, a ReLU and the first linear layer. For instance, if we want the gradients of the loss with respect to `b2` and our loss is defined by:

```
loss = mse(out,y) = mse(lin(l2, w2, b2), y)
```

The chain rule tells us that we have:
$$\frac{\text{d} loss}{\text{d} b_{2}} = \frac{\text{d} loss}{\text{d} out} \times \frac{\text{d} out}{\text{d} b_{2}} = \frac{\text{d}}{\text{d} out} mse(out, y) \times \frac{\text{d}}{\text{d} b_{2}} lin(l_{2}, w_{2}, b_{2})$$

To compute the gradients of the loss with respect to $b_{2}$, we first need the gradients of the loss with respect to our output $out$. It's the same if we want the gradients of the loss with respect to $w_{2}$. Then, to get the gradients of the loss with respect to $b_{1}$ or $w_{1}$, we will need the gradients of the loss with respect to $l_{1}$, which in turn requires the gradients of the loss with respect to $l_{2}$, which will need the gradients of the loss with respect to $out$.

So to compute all the gradients we need for the update, we need to begin from the output of the model and work our way *backward*, one layer after the other—which is why this step is known as *backpropagation*. We can automate it by having each function we implemented (`relu`, `mse`, `lin`) provide its backward step: that is, how to derive the gradients of the loss with respect to the input(s) from the gradients of the loss with respect to the output.

Here we populate those gradients in an attribute of each tensor, a bit like PyTorch does with `.grad`. 

The first are the gradients of the loss with respect to the output of our model (which is the input of the loss function). We undo the `squeeze` we did in `mse`, then we use the formula that gives us the derivative of $x^{2}$: $2x$. The derivative of the mean is just $1/n$ where $n$ is the number of elements in our input:

In [127]:
def mse_grad(inp, targ):
  inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1)/ inp.shape[0]

In [128]:
def relu_grad(inp, out):
  inp.g = (inp>0).float() *out.g

In [129]:
def lin_grad(inp, out, w, b):
  inp.g = out.g @ w.t()
  w.g = inp.t() @ out.g 
  b.g = out.g.sum(0)

## Sympy

for doing symboliccalculations

In [131]:
from sympy import symbols, diff
sx, sy = symbols('sx sy')
diff(sx ** 2, sx)

2*sx

In [132]:
def forward_and_backward(inp, targ):
  l1 = inp @ w1 + b1
  l2 = relu(l1)
  out = l2 @ w2 + b2
  loss = mse(out, targ)

  mse_grad(out, targ)
  lin_grad(l2, out, w2, b2)
  relu_grad(l1, l2)
  lin_grad(inp, l1, w1, b1)

## refactoring the model

In [162]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

`__call__` is a magic name in Python that will make our class callable. This is what will be executed when we type `y = Relu()(x)`. We can do the same for our linear layer and the MSE loss:

In [155]:
class Lin():
  def __init__(self, w, b):
    self.w, self.b = w, b
  
  def __call__(self, inp):
    self.inp = inp
    self.out = inp@self.w + self.b
    return self.out
  
  def backward(self):
    self.inp.g = self.out.g @ self.w.t()
    self.w.g = self.inp.t() @ self.out.g
    self.b.g = self.out.g.sum(0)

In [166]:
class Mse():
  def __call__(self, inp, targ):
    self.inp = inp
    self.targ = targ
    self.out = (inp.squeeze() - targ).pow(2).mean()
    return self.out
  
  def backward(self):
    x = (self.inp.squeeze()-self.targ).unsqueeze(-1)
    self.inp.g  = 2.*x/self.targ.shape[0]
    

In [156]:
class Model():
  def __init__(self, w1, b1, w2, b2):
    self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2) ]
    self.loss = Mse()

  def __call__(self, x, targ):
    for l in self.layers: x = l(x)
    return self.loss(x, targ)
  
  def backward(self):
    self.loss.backward()
    for l in reversed(self.layers):
      l.backward()
      

In [167]:
model = Model(w1, b1, w2, b2)

In [168]:
loss = model(x, y) # forward pass

In [169]:
model.backward() # backward pass

## Going to pytorch

In [170]:
class LayerFunction():
  def __call__(self, *args):
    self.args = args
    self.out = self.forward(*args)
    return self.out
  
  def forward(self):
    raise Exception('not implemented')
  def bwd(self):
    raise Exception('not implemented')
  def backward(self):
    self.bwd(self.out, *self.args)

In [171]:
# lets just implement the layer functionfor creating varios layers

class Relu(LayerFunction):
  def forward(self, inp):
    return inp.clamp_min(0.)
  def bwd(self, out, inp):
    inp.g = (inp>0).float() * out.g

In [ ]:
class Lin(LayerFunction):
  def __init__(self, w, b):
    self.w, self.b = w, b
  
  def forward(self, inp):
    return inp@self.w  + self.b
  
  def bwd(self, out, inp):
    inp.g = out.g @self.w.t()
    self.w.g = self.inp.t() @ slef.out.g
    self.b.g = out.g.sum(0)
  
   

In [172]:
class Mse(LayerFunction):
  def forward( self, inp, targ):
    return (inp.squeeze() - targ).pow(2).mean()
  
  def bwd(self, out, inp, targ):
    inp.g = 2 * (inp.squeeze() - targ).unsqueeze(-1)/targ.shape[0]

The rest of our model can be the same as before. This is getting closer and closer to what PyTorch does. Each basic function we need to differentiate is written as a `torch.autograd.Function` object that has a `forward` and a `backward` method. PyTorch will then keep trace of any computation we do to be able to properly run the backward pass, unless we set the `requires_grad` attribute of our tensors to `False`.

Writing one of these is (almost) as easy as writing our original classes. The difference is that we choose what to save and what to put in a context variable (so that we make sure we don't save anything we don't need), and we return the gradients in the `backward` pass. It's very rare to have to write your own `Function` but if you ever need something exotic or want to mess with the gradients of a regular function, here is how to write one:

In [173]:
from torch.autograd import Function

class MyRelu(Function):
  @staticmethod
  def forward(ctx, i):
    result = i.clamp_min(0.)
    ctx.save_for_backward(i)
    return result
  
  @staticmethod
  def backward(ctx, grad_output):
    i, = ctx.saved_tensors
    return grad_output * (i>0).float()

The structure used to build a more complex model that takes advantage of those `Function`s is a `torch.nn.Module`. This is the base structure for all models, and all the neural nets you have seen up until now were from that class. It mostly helps to register all the trainable parameters, which as we've seen can be used in the training loop.

To implement an `nn.Module` you just need to:

- Make sure the superclass `__init__` is called first when you initialize it.
- Define any parameters of the model as attributes with `nn.Parameter`.
- Define a `forward` function that returns the output of your model.

As an example, here is the linear layer from scratch:

In [174]:
import torch.nn as nn

class LinearLayer(nn.Module):
  def __init__(self, n_in, n_out):
    super().__init__()
    self.weight = nn.Parameter(torch.randn(n_out,n_in) * sqrt(2/n_in))
    self.bias = nn.Parameter(torch.zeros(n_out))
  
  def forward(self, x):
    return x @ self.weight.t() + self.bias

In [175]:
lin = LinearLayer(10,2)
p1,p2 = lin.parameters()
p1.shape, p2.shape

(torch.Size([2, 10]), torch.Size([2]))

It is thanks to this feature of `nn.Module` that we can just say `opt.step()` and have an optimizer loop through the parameters and update each one.

Note that in PyTorch, the weights are stored as an `n_out x n_in` matrix, which is why we have the transpose in the forward pass.

By using the linear layer from PyTorch (which uses the Kaiming initialization as well), the model we have been building up during this chapter can be written like this:

In [176]:
class Model(nn.Module):
  def __init__(self, n_in,nh, n_out):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Linear(n_in, nh), nn.ReLU(), nn.Linear(nh, n_out)
    )
    self.loss = mse
  
  def forward(self, x, targ):
    return self.loss(self.layers(x).squeeze(), targ)

fastai provides its own variant of `Module` that is identical to `nn.Module`, but doesn't require you to call `super().__init__()` (it does that for you automatically):

In [179]:
from fastai.basics import Module

In [180]:
class Model(Module):
  def __init__(self, n_in, nh, n_out):
    self.layers = nn.Sequential(
        nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh, n_out)
    )
    self.loss = mse
  
  def forward(self, x, targ):
    return self.loss(self.layers(x).squeeze(), targ)

# we will use it next chapter